In [1]:
!pip install nltk gensim

In [2]:
"""import json
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


def predict_NuExtract(model, tokenizer, text, schema, example=["","",""]):
    schema = json.dumps(json.loads(schema), indent=4)
    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"
    for i in example:
      if i != "":
          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"
    
    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"
    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")

    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)
    return output.split("<|output|>")[1].split("<|end-output|>")[0]


model = AutoModelForCausalLM.from_pretrained("numind/NuExtract", trust_remote_code=True, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("numind/NuExtract", trust_remote_code=True)

model.to("cuda")

model.eval()

text = """"""We introduce Mistral 7B, a 7–billion-parameter language model engineered for
superior performance and efficiency. Mistral 7B outperforms the best open 13B
model (Llama 2) across all evaluated benchmarks, and the best released 34B
model (Llama 1) in reasoning, mathematics, and code generation. Our model
leverages grouped-query attention (GQA) for faster inference, coupled with sliding
window attention (SWA) to effectively handle sequences of arbitrary length with a
reduced inference cost. We also provide a model fine-tuned to follow instructions,
Mistral 7B – Instruct, that surpasses Llama 2 13B – chat model both on human and
automated benchmarks. Our models are released under the Apache 2.0 license.
Code: https://github.com/mistralai/mistral-src
Webpage: https://mistral.ai/news/announcing-mistral-7b/""""""

schema = """"""{
    "Model": {
        "Name": "",
        "Number of parameters": "",
        "Number of token": "",
        "Architecture": []
    },
    "Usage": {
        "Use case": [],
        "Licence": ""
    }
}""""""

prediction = predict_NuExtract(model, tokenizer, text, schema, example=["","",""])
print(prediction)
"""

'import json\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\nimport torch\n\n\ndef predict_NuExtract(model, tokenizer, text, schema, example=["","",""]):\n    schema = json.dumps(json.loads(schema), indent=4)\n    input_llm =  "<|input|>\n### Template:\n" +  schema + "\n"\n    for i in example:\n      if i != "":\n          input_llm += "### Example:\n"+ json.dumps(json.loads(i), indent=4)+"\n"\n    \n    input_llm +=  "### Text:\n"+text +"\n<|output|>\n"\n    input_ids = tokenizer(input_llm, return_tensors="pt", truncation=True, max_length=4000).to("cuda")\n\n    output = tokenizer.decode(model.generate(**input_ids)[0], skip_special_tokens=True)\n    return output.split("<|output|>")[1].split("<|end-output|>")[0]\n\n\nmodel = AutoModelForCausalLM.from_pretrained("numind/NuExtract", trust_remote_code=True, torch_dtype=torch.bfloat16)\ntokenizer = AutoTokenizer.from_pretrained("numind/NuExtract", trust_remote_code=True)\n\nmodel.to("cuda")\n\nmodel.eval()\n\ntext = We int

In [3]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [4]:
df_reports = pd.read_csv("preprocessed_reports.csv")
df_paragraphs = pd.read_csv("preprocessed_paragraphs.csv")

In [5]:
df_paragraphs_testing = df_paragraphs

In [6]:
df_paragraphs_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342 entries, 0 to 341
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  342 non-null    int64 
 1   ID          342 non-null    int64 
 2   paragraphs  342 non-null    object
dtypes: int64(2), object(1)
memory usage: 8.1+ KB


In [7]:
df_paragraphs_testing.value_counts("ID")

ID
1     53
5     44
18    42
15    31
12    23
9     19
8     17
10    14
4     13
17    13
20    13
16    11
2     10
3      9
7      7
6      7
11     6
13     6
14     4
Name: count, dtype: int64

In [8]:
def clean_text_again(text):
    text = strip_tags(text)
    text = strip_numeric(text)
    text = strip_punctuation(text)  # Remove punctuation
    text = strip_multiple_whitespaces(text)  # Normalize whitespaces
    text = remove_stopwords(text)  # Remove stopwords
    return text

In [9]:
#df_paragraphs_testing["paragraphs"] = df_paragraphs_testing["paragraphs"].apply(clean_text_again)

# FINBERT

# KeyBERT

In [10]:
"""import pandas as pd
from keybert import KeyBERT

# Initialize KeyBERT model
kw_model = KeyBERT()

# Function to extract keywords or key phrases
def extract_keywords(text, ngram_range=(1, 4), top_n=5):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=ngram_range, stop_words='english', top_n=top_n)
    # Return just the keywords (without similarity scores)
    return [kw for kw in keywords]"""

"import pandas as pd\nfrom keybert import KeyBERT\n\n# Initialize KeyBERT model\nkw_model = KeyBERT()\n\n# Function to extract keywords or key phrases\ndef extract_keywords(text, ngram_range=(1, 4), top_n=5):\n    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=ngram_range, stop_words='english', top_n=top_n)\n    # Return just the keywords (without similarity scores)\n    return [kw for kw in keywords]"

# LLM Argument Mining

- Provide List of pre-defined arguments

In [11]:
list_of_arguments = [
    "Cost Management",
    "Revenue Growth",
    "Cash Management",
    "Market Position",
    "Growth Strategy",
    "Financial Performance",
    "Capital Management",
    "Product Strategy",
    "Supply Chain",
    "Regulatory Environment",
    "Dividend Policy",
    "Mergers and Acquisitions",
    "Technology",
    "Customer Engagement",
    "Market Trends",
    "Earnings Guidance",
    "Investment Strategy",
    "Pricing Strategy",
    "Geographic Performance",
    "Balance Sheet",
    "Corporate Governance",
    "Operational Performance",
    "Risk Management",
    "Research and Development",
    "Market Performance",
    "Growth Drivers",
    "Financial Guidance",
    "Brand Strategy",
    "E-commerce",
    "Sustainability",
    "Shares Repurchase",
    "Competitive Environment",
    "Operational Risks",
    "Financial Health",
    "Sales Strategy",
    "Macroeconomic Impact",
    "Innovation"
]


In [15]:
#list_of_arguments = ["Financial Performance","Revenue Growth"]
# paragraph removed from instructions: - New categories must be generic, concise (max 3 words), and included in the output for list updates.
"""You will classify financial text into predefined categories or create a new one if needed:
- Assign arguments to the best-matching category. Create a new category only if no match fits.
- IGNORE brand names/Target Prices/Recommendations/Valuation as arguments.
- Classify each argument as positive, negative, or neutral.
- Include new categories in the output.
- Return only the JSON dictionary, no extra text.

In system_content: Categories: [{categories_string}]
"""


def find_arguments(text, api_key):
    client = Groq(api_key=api_key)

    categories_string = ', '.join(f'"{cat}"' for cat in list_of_arguments)

    system_content = f"""
You will classify arguments in financial text into categories:
- Categories should be concises (maximum 3 words)
- Assign arguments to best-matching category. Create a new category only if no other category fits.
- IGNORE brand names/Target Prices/Recommendations/Valuation as arguments.
- Classify each argument as positive, negative, or neutral.
- Output a JSON dictionary with arguments as keys and their sentiment as values (e.g., {{"Cost Management": "positive"}}).
- Return only the JSON dictionary, no extra text."""
    
    final_prompt = f"""
        Text: {text}
       """

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_content,
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama3-70b-8192",
    )

    print(f"Response: {response}")

    return response.choices[0].message.content

In [16]:
# Load API key
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

def find_arguments_with_delay(commentary):
    global list_of_arguments  # Ensure we can modify the global categories list
    result_json = find_arguments(commentary, api_key)
    
    try:
        # Parse the JSON response
        result_dict = json.loads(result_json)
        
        # Check for new categories and add them to the global list
        for category in result_dict.keys():
            if category not in list_of_arguments:
                list_of_arguments.append(category)  # Dynamically update the categories list
        
        return result_dict  # Return the result
    except json.JSONDecodeError:
        print("Failed to decode JSON response.")
        return None


In [17]:
df_paragraphs_testing["provided arguments"] = df_paragraphs_testing["paragraphs"].apply(find_arguments_with_delay)

Response: ChatCompletion(id='chatcmpl-f153dba7-125d-4e15-8870-72e35a005f33', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"Solid 1QF20": "positive", "Long-term Growth": "positive", "Channel Development": "positive", "Positive Traffic": "positive", "Buying Opportunity": "positive", "Growth at Scale": "positive", "Material Disruption": "negative", "Traffic Pressure": "negative", "Business Disruption": "negative", "Financial Impact": "negative", "Uncertainty": "neutral", "Caution": "neutral"}', role='assistant', function_call=None, tool_calls=None))], created=1732449337, model='llama3-70b-8192', object='chat.completion', system_fingerprint='fp_753a4aecf6', usage=CompletionUsage(completion_tokens=94, prompt_tokens=601, total_tokens=695, completion_time=0.268571429, prompt_time=0.03286653, queue_time=0.007966479000000005, total_time=0.301437959), x_groq={'id': 'req_01jdf0e08ce75rq7216rds5qp8'})
Response: ChatCompletion(id='chatcmpl-1d

In [160]:
df_paragraphs_testing.head()

,Unnamed: 0,ID,paragraphs,provided arguments
10,10,2,Salesforce reported excellent F1Q17 results an...,"{'Revenue Growth': 'positive', 'Earnings Guida..."
11,11,2,Excellent results. Revenues of 2.39B ahead of ...,"{'Revenue Growth': 'positive', 'Financial Perf..."
12,12,2,Sum is more than the parts. A rare case of the...,"{'Growth Strategy': 'positive', 'Product Strat..."
13,13,2,"Valuation Numbers change little, aside from sh...","{'Earnings Guidance': 'neutral', 'Financial Gu..."
14,14,2,"SALESFORCE.COM, INC. CRM, Buy, 100.00 PT Curre...",None


In [18]:
#df_paragraphs_testing.to_csv("without_data_cleaning_test.csv")

In [19]:
import pandas as pd
import ast

# Initialize a list to store the exploded rows
rows = []

# Iterate through the DataFrame and expand the dictionary into rows
for index, row in df_paragraphs_testing.iterrows():
    provided_args = row["provided arguments"]
    # Convert string to dictionary if needed
    if isinstance(provided_args, str):
        try:
            provided_args = ast.literal_eval(provided_args)  # Convert string to dictionary
        except (ValueError, SyntaxError):
            print(f"Invalid format for provided_args at index {index}: {provided_args}")
            continue  # Skip invalid entries

    # Ensure it's a dictionary
    if isinstance(provided_args, dict):
        #print(f"Processed provided_args: {provided_args}")
        for argument, sentiment in provided_args.items():
            rows.append({
                "ID": row["ID"],
                "Argument": argument,
                "Sentiment": sentiment
            })

# Create a new DataFrame from the exploded rows
df_result = pd.DataFrame(rows)

# Display the resulting DataFrame
print(df_result)

     ID             Argument Sentiment
0     1          Solid 1QF20  positive
1     1     Long-term Growth  positive
2     1  Channel Development  positive
3     1     Positive Traffic  positive
4     1   Buying Opportunity  positive
..   ..                  ...       ...
845  20     Market Expansion  positive
846  20        US EV credits  positive
847  20   raw material costs  negative
848  20        US regulation  negative
849  20      Model 3Y demand  negative

[850 rows x 3 columns]


In [20]:
df_result.to_csv("list_of_arguments3.csv")

In [15]:
df.info()

NameError: name 'df' is not defined

In [46]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_paragraphs_testing["provided arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
5,Financial Guidance,6,2,0
0,Financial Performance,5,2,0
6,Share Repurchase,5,1,0
24,Revenue Growth,5,1,0
2,Capital Management,4,0,0
1,Dividend Policy,3,1,1
3,Earnings Guidance,3,1,0
10,Operational Efficiency,3,1,0
31,Product Strategy,3,1,0
32,Growth Strategy,3,0,0


# Check amount of intensifiers

In [47]:

# Basic Intensifiers
basic_intensifiers = [
    "very", "really", "quite", "so", "too", "totally", "absolutely",
    "completely", "utterly", "entirely", "truly", "fully", "genuinely", "surely"
]

# Strong Intensifiers (includes previous stronger, superlative, and "all/full" intensifiers)
strong_intensifiers = [
    "extremely", "incredibly", "highly", "immensely", "vastly", "remarkably", 
    "tremendously", "phenomenally", "exceptionally", "noticeably", "supremely", 
    "powerfully", "acutely", "enormously", "greatly", "immeasurably", "infinitely",
    "wildly", "lavishly", "ludicrously", "fiercely", "ferociously", "intensely",
    "extraordinarily", "overwhelmingly", "staggeringly", "excessively", "insanely", 
    "unbelievably", "inconceivably", "ridiculously", "crazily", "all-out", 
    "all-around", "full-on", "fully", "flat-out"
]

# Formal/Emotive Intensifiers (combines emotive and formal language for a more academic or dramatic tone)
formal_emotive_intensifiers = [
    "profoundly", "considerably", "significantly", "markedly", "appreciably", 
    "notably", "exceedingly", "far", "deeply", "awfully", "horribly", "dreadfully", 
    "frightfully", "painfully", "absurdly", "scandalously", "outrageously", 
    "shockingly", "horrendously", "terribly", "hideously", "grossly"
]


In [48]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [51]:
# Apply the function to the "Sentence" column of the DataFrame
df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=strong_intensifiers)
df_paragraphs_testing["Formal Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=formal_emotive_intensifiers)

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifie

In [52]:
df_paragraphs_testing

,Unnamed: 0,ID,paragraphs,provided arguments,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,0,1,We maintain our Overweight rating and raise ou...,"{'Financial Performance': 'positive', 'Dividen...",0,0,0
1,1,1,We remain in the minority with less than a thi...,"{'Balance Sheet': 'negative', 'Dividend Policy...",1,0,0
2,2,1,"Adjustments reect Q3 2021 actuals, our updated...","{'Operational Efficiency': 'positive', 'Capita...",0,0,0
3,3,1,XOM expects upstream volumes to increase on lo...,"{'Operational Efficiency': 'neutral', 'Upstrea...",0,0,0
4,4,1,XOM announced a 10bn share repurchase Buybacks...,"{'Share Repurchase': 'positive', 'Dividend Pol...",0,0,0
5,5,1,XOM announced annual capex range of 2025bn goi...,"{'Capital Management': 'positive', 'Cost Struc...",1,1,0
6,6,1,EPS of 1.57 was in betweenour estimate of 1.58...,"{'Financial Performance': 'neutral', 'Operatio...",0,0,0
7,7,1,Our 75 price target is based on 6.5x our 2023 ...,"{'Valuation': 'neutral', 'Earnings Guidance': ...",0,0,0
8,8,1,Systematic risks to our targetrating include c...,"{'Regulatory Environment': 'negative', 'Operat...",0,0,0
9,9,1,XOM We have an Overweight rating on XOM. Opex ...,"{'Financial Performance': 'positive', 'Cost Ma...",0,0,0
